In [ ]:
import re
import pandas as pd

df = pd.read_csv('../data/2_ds2_trimmed.csv')
df = df.dropna(subset=['tag', 'lyrics'])

In [110]:
# Used this to test different chunks of data

import pandas as pd
column_names = pd.read_csv(file_path, nrows=0).columns

chunk_size = 10**5  # Define your chunk size
chunk_number = 3     # Specify which chunk you want (e.g., 2nd chunk here)

# Calculate the number of rows to skip
rows_to_skip = chunk_size * (chunk_number - 1)+1

# Load the next chunk with skiprows
df = pd.read_csv(file_path, skiprows=rows_to_skip, nrows=chunk_size,  header=None,names=column_names)
df['lyrics']=df['lyrics'].astype(str)
# Display a sample to verify
df.head()

,title,tag,artist,year,views,features,lyrics,id
0,Flame Upgrade-Yo Prod.,rap,X-RATE PRO (El-Pablo),2013,6,"{""PM (FAM)""}","Check tha TRAP track from X-RATE PRO, comment ...",213825
1,DYou Know What I Mean?,rock,Oasis,1997,51628,{},[Intro]\n\n[Verse 1]\nStep off the train all a...,213826
2,Dunkelheit,rap,Disarstar,2011,1043,{},[Intro]\nRap ist Therapie für mich\nSonst dreh...,213828
3,La loi du plus fort,rap,Sinik,2011,1035,{},"O.K., au pied des tours, sa part en couilles p...",213829
4,Purposefully Upgrade-Yo Prod.,rap,X-RATE PRO,2013,5,{},"Check tha dirty-south track, comment & share, ...",213830


In [111]:
df.head()

,title,tag,artist,year,views,features,lyrics,id
0,Flame Upgrade-Yo Prod.,rap,X-RATE PRO (El-Pablo),2013,6,"{""PM (FAM)""}","Check tha TRAP track from X-RATE PRO, comment ...",213825
1,DYou Know What I Mean?,rock,Oasis,1997,51628,{},[Intro]\n\n[Verse 1]\nStep off the train all a...,213826
2,Dunkelheit,rap,Disarstar,2011,1043,{},[Intro]\nRap ist Therapie für mich\nSonst dreh...,213828
3,La loi du plus fort,rap,Sinik,2011,1035,{},"O.K., au pied des tours, sa part en couilles p...",213829
4,Purposefully Upgrade-Yo Prod.,rap,X-RATE PRO,2013,5,{},"Check tha dirty-south track, comment & share, ...",213830


In [112]:
def clean_lyrics(text):
    # Remove all text within square brackets
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\{.*?\}', '', text)
    text = re.sub(r'\(.*?\)', '', text)
    # Remove any remaining newline characters
    text = text.replace('\n', ' ')
    # Remove leading/trailing whitespace
    text = text.strip()
    return text

In [113]:
# clean out nonuseful words
df['lyrics']=df['lyrics'].apply(clean_lyrics)
# Keep only lyrics with 20 or more words
clean_df = df[df['lyrics'].str.split().str.len() >= 20]

# note gets rid of short songs, zero lyrics, intrumentals, and non lric words such as intro chorus etc

clean_df.head()

,title,tag,artist,year,views,features,lyrics,id
1,DYou Know What I Mean?,rock,Oasis,1997,51628,{},Step off the train all alone at dawn Back into...,213826
2,Dunkelheit,rap,Disarstar,2011,1043,{},Rap ist Therapie für mich Sonst dreh ich durch...,213828
3,La loi du plus fort,rap,Sinik,2011,1035,{},"O.K., au pied des tours, sa part en couilles p...",213829
5,Parents,rap,Denzel Curry,2013,73510,{},"Your momma ain't shit, your daddy ain't shit A...",213831
6,Miroir sans teint,rap,Ol' Kainry,2010,1140,"{Sinik,""Ol\\' Kainry""}",J'ai pris ma plume pour assumer La race humain...,213832


In [103]:
# Sample of these short lyrics to inspect
#sampled_short_lyrics = short_lyrics.sample(15, random_state=15)

# Display the results
#for index, lyric in enumerate(sampled_short_lyrics['lyrics'], 1):
#    print(f"Lyrics Sample {index}:\n{lyric}\n{'-'*50}")


In [114]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensure consistent results from langdetect
DetectorFactory.seed = 0

def is_english(text):
    try:
        # Detect language of text
        return detect(text) == 'en'
    except LangDetectException:
        # If language detection fails, treat it as non-English
        return False


In [109]:
# #This was a check to see if it sorted correctly
# # Note this can take a long time with large inputs

# # Sample 100 random lyrics along with titles
# sampled_lyrics = df[['title', 'lyrics']].sample(100, random_state=5)

# # Check each sampled lyric for English language
# sampled_lyrics['is_english'] = sampled_lyrics['lyrics'].apply(is_english)

# # Filter for non-English lyrics in the sampled set
# non_english_lyrics = sampled_lyrics[sampled_lyrics['is_english'] == False]

# # Display non-English sampled lyrics
# print("Sampled Lyrics Detected as Non-English:")
# for index, row in non_english_lyrics.iterrows():
#     print(f"Title: {row['title']}\nLyrics:\n{row['lyrics']}\n{'-'*50}")


In [ ]:
# note this can take a very large time recommend trying your computer with a sample first.

clean_df = clean_df[clean_df['lyrics'].apply(is_english)]


In [ ]:
clean_df.to_csv("../data/3_ds3_cleaned.csv", index=False)
